Script to create a list of Apex triggers across all objects, along with status and trigger contexts

In [9]:
import os
import pandas as pd
import xml.etree.ElementTree as ET
import re

# Set the directory path; path below assumes that this project is in the same directory as the Salesforce project 
directory_path = '../projectName/force-app/main/default/triggers/'

# Create an empty DataFrame
df = pd.DataFrame(columns=['Object', 'Name', 'Status', 'TriggerContexts'])

# Loop through each trigger file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".trigger"):
        # Create new row for dataframe
        trigger_data = {}
        
        # Open the trigger file and read its contents
        with open (directory_path + filename, 'r') as file:
            lines = file.readlines()
            # Iterate over each line
            for line in lines:
                # Check if the line starts with "trigger" and parse the relevant data
                if line.strip().lower().startswith("trigger"):
                    words = re.split(r'\s|\(', line)
                    trigger_data['Object'] = words[3] if len(words) > 3 else ''
                    trigger_data['Name'] = words[1] if len(words) > 1 else ''
                    context_start = line.find('(') + 1
                    context_end = line.find(')')
                    trigger_data['TriggerContexts'] = line[context_start:context_end] if context_start < context_end else ''

        # Get the trigger status
        trigger_meta_filename = filename + '-meta.xml'
        file_path = os.path.join(directory_path, trigger_meta_filename)

        # Load the XML file
        tree = ET.parse(file_path)
        root = tree.getroot()
        trigger_data['Status'] = root.find('{http://soap.sforce.com/2006/04/metadata}status').text

        # Append the data to the list
        df.loc[len(df)] = trigger_data

# Display the DataFrame
print(df)

df.to_csv('triggers.csv', index=False)

                                Object                      Name    Status  \
0           MailChimpSyncProcessing__e   MailChimpPostProcessing    Active   
1                 Course_Enrollment__c   CourseEnrollmentTrigger    Active   
2   CohortOfferingStudentEnrollment__c               UpdateCoach    Active   
3                 CourseCohortEvent__c      CohortSessionTrigger  Inactive   
4                 Course_Enrollment__c  UpdateDateChangeProgress  Inactive   
5                              Contact            ContactTrigger    Active   
6                       Interaction__c           INT_Interaction    Active   
7                                 Task               TaskTrigger    Active   
8   CohortOfferingStudentEnrollment__c          UpdateCohortDate    Active   
9                              Contact            updateCheckBox    Active   
10                Course_Enrollment__c             UpdatePACABox  Inactive   
11                             Contact      ContactTriggerMaster